# One sample test on simulated bulk RNA-seq

Nuha BinTayyash, 2020

This notebook shows how to run GPcounts with one sample test on simulated bulk RNA-seq datasets.

In [1]:
import numpy as np
import pandas as pd
from GPcounts.GPcounts_Module import Fit_GPcounts
import gpflow

In [2]:
files = ['low_counts_low_dispersion_non_differentially_expressed_genes_DE_non_DE_genes.csv'
         ,'low_counts_high_dispersion_non_differentially_expressed_genes_DE_non_DE_genes.csv'                                      
         ,'high_counts_low_dispersion_non_differentially_expressed_genes_DE_non_DE_genes.csv'
         ,'high_counts_high_dispersion_non_differentially_expressed_genes_DE_non_DE_genes.csv']

X = pd.read_csv('time_points.csv',index_col=[0])

for file in files:
    print(file)
    Y = pd.read_csv(file,index_col=[0])
    gp_counts = Fit_GPcounts(X,Y) 
    likelihoods = ['Gaussian'] #'Negative_binomial',,'Poisson'
    for likelihood in likelihoods:
        log_likelihood = gp_counts.One_sample_test(likelihood)
        log_likelihood.to_csv("ll_"+likelihood+"_"+file)

  0%|          | 0/358 [00:00<?, ?it/s]

low_counts_low_dispersion_non_differentially_expressed_genes_DE_non_DE_genes.csv
local Optima
y_mean 0.8564523225172295
mean_mean 0.9367562513634258
abs(round((mean_mean-y_mean)/y_mean)) 0.0


  0%|          | 1/358 [00:06<37:29,  6.30s/it]

local Optima
y_mean 0.8564523225172295
mean_mean 0.7982191313837407
abs(round((mean_mean-y_mean)/y_mean)) 0.0
local Optima
y_mean 3.120775090879673
mean_mean 3.1206784127690623
abs(round((mean_mean-y_mean)/y_mean)) 0.0


  1%|          | 2/358 [00:07<29:04,  4.90s/it]

...

local Optima
y_mean 9.995832454177911
mean_mean 9.995108525278226
abs(round((mean_mean-y_mean)/y_mean)) 0.0
local Optima
y_mean 9.995832454177911
mean_mean 9.995105514270021
abs(round((mean_mean-y_mean)/y_mean)) 0.0
local Optima
y_mean 7.453833347658965
mean_mean 7.092301245773167
abs(round((mean_mean-y_mean)/y_mean)) 0.0


100%|█████████▉| 359/360 [02:32<00:00,  2.02it/s]

local Optima
y_mean 7.453833347658965
mean_mean 7.431695728470482
abs(round((mean_mean-y_mean)/y_mean)) 0.0


100%|██████████| 360/360 [02:32<00:00,  2.36it/s]

local Optima
y_mean 10.010943744887905
mean_mean 10.01047777602939
abs(round((mean_mean-y_mean)/y_mean)) 0.0
local Optima
y_mean 10.010943744887905
mean_mean 10.010202259233342
abs(round((mean_mean-y_mean)/y_mean)) 0.0


In [3]:
log_likelihood

,Dynamic_model_log_likelihood,Constant_model_log_likelihood,log_likelihood_ratio
gene_1,-37.574788,-42.269266,4.694478
gene_2,-21.127475,-21.127412,-0.000062
gene_3,-37.914913,-37.508437,-0.406476
gene_4,-20.675636,-20.675634,-0.000003
gene_5,-34.471219,-42.191281,7.720062
...,...,...,...
gene_356,-20.323807,-20.494668,0.170861
gene_357,-39.783372,-42.442096,2.658723
gene_358,-11.997632,-11.997737,0.000105
gene_359,-35.116495,-38.247876,3.131381
